In [ ]:
import numpy as np
import pandas as pd

In [ ]:
books=pd.read_csv('Books.csv')
users=pd.read_csv('Users.csv')
ratings=pd.read_csv('Ratings.csv')

Finding the total Null values and duplicated in each datasets

In [ ]:
print(books.isnull().sum())
print(ratings.isnull().sum())
print(users.isnull().sum())

In [ ]:
print(books.duplicated().sum())
print(users.duplicated().sum())
print(ratings.duplicated().sum())

*Popularity Based Recommender System*

In [23]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [24]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

,Book-Title,num_ratings
0,Earth Prayers From around the World: 365 Pray...,10
1,Final Fantasy Anthology: Official Strategy Gu...,4
2,Flight of Fancy: American Heiresses (Zebra Ba...,2
3,Little Comic Shop of Horrors (Give Yourself G...,4
4,Mystery Mile,2
...,...,...
68750,Ã?Â?lpiraten.,2
68751,Ã?Â?rger mit Produkt X. Roman.,4
68752,Ã?Â?sterlich leben.,1
68753,Ã?Â?stlich der Berge.,3


In [25]:
avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_df

,Book-Title,avg_rating
0,Earth Prayers From around the World: 365 Pray...,5.000000
1,Final Fantasy Anthology: Official Strategy Gu...,5.000000
2,Flight of Fancy: American Heiresses (Zebra Ba...,4.000000
3,Little Comic Shop of Horrors (Give Yourself G...,1.250000
4,Mystery Mile,0.000000
...,...,...
68750,Ã?Â?lpiraten.,0.000000
68751,Ã?Â?rger mit Produkt X. Roman.,5.250000
68752,Ã?Â?sterlich leben.,7.000000
68753,Ã?Â?stlich der Berge.,2.666667


In [26]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_rating
0,Earth Prayers From around the World: 365 Pray...,10,5.000000
1,Final Fantasy Anthology: Official Strategy Gu...,4,5.000000
2,Flight of Fancy: American Heiresses (Zebra Ba...,2,4.000000
3,Little Comic Shop of Horrors (Give Yourself G...,4,1.250000
4,Mystery Mile,2,0.000000
...,...,...,...
68750,Ã?Â?lpiraten.,2,0.000000
68751,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
68752,Ã?Â?sterlich leben.,1,7.000000
68753,Ã?Â?stlich der Berge.,3,2.666667


In [27]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)

In [28]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

In [29]:
popular_df['Image-URL-M'][0]

'http://images.amazon.com/images/P/043935806X.01.MZZZZZZZ.jpg'

## Collaborative Filtering Based Recommender System

In [38]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
g_users = x[x].index

In [39]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(g_users)]

In [40]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [41]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [42]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [43]:
pt.fillna(0,inplace=True)

In [44]:
pt

User-ID
Book-Title


In [45]:
from sklearn.metrics.pairwise import cosine_similarity


In [47]:
similarity_scores = cosine_similarity(pt)

ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required by check_pairwise_arrays.